In [42]:
import joblib
import pdfplumber
import re
import spacy
import numpy as np

In [43]:
# Load the saved model and vectorizer
tfidf = joblib.load('/Users/macbookair/Desktop/project/Resume_Analyser/resume-job-matching/models/tfidf_vectorizer.pkl')
best_model = joblib.load('/Users/macbookair/Desktop/project/Resume_Analyser/resume-job-matching/models/gradient_boosting_model.pkl')

In [44]:
nlp = spacy.load('en_core_web_sm')

In [45]:
def cleanText(text):
    text = re.sub(r'[^a-zA-Z0-9\s+]', '', text) 
    text = re.sub(r'\S+@\S+', '', text) 
    text = re.sub(r'\d{10,}', '', text)
    text = re.sub('\s+', ' ', text)
    return text.lower()

In [46]:
def lemmatize_tokens(tokens):
    doc = nlp(' '.join(tokens))
    return [token.lemma_ for token in doc]

In [47]:
def remove_stopwords_from_tokens(tokens):
    stop_words = set(nlp.Defaults.stop_words)
    return [word for word in tokens if word.lower() not in stop_words]

In [48]:
def extract_text_from_pdf(pdf_path):
    """Extract text from a single PDF file."""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            full_text = ''
            for page in pdf.pages:
                full_text += page.extract_text() or ''  # Extract text from all pages
            return full_text.strip()  # Clean up whitespace
    except Exception as e:
        print(f"Error reading {pdf_path}: {e}")
        return ''

In [49]:
resume = '/Users/macbookair/Desktop/project/Resume_Analyser/resume-job-matching/data/Resume/resume.pdf'

In [50]:
extracted_text = extract_text_from_pdf(resume)
cleaned_resume = cleanText(extracted_text)


In [51]:
tokens = cleaned_resume.split()
tokens = remove_stopwords_from_tokens(tokens)
lemmatized_tokens = lemmatize_tokens(tokens)

In [52]:
final_resume_text = ' '.join(lemmatized_tokens)

In [53]:
resume_vectorized = tfidf.transform([final_resume_text])

In [54]:
prediction = best_model.predict(resume_vectorized)

NotFittedError: This GradientBoostingClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.